In [6]:
pip install selenium


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.4 MB 333 kB/s eta 0:00:01
     |████████████████████████████████| 121 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 475 kB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 4.7 MB/s eta 0:00:011
     |████████████████████████████████| 66 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install moviepy speechrecognition opencv-python pytesseract

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 388 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 32.8 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 54.8 MB 2.5 MB/s eta 0:00:012
     |████████████████████████████████| 78 kB 5.4 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 313 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 4.6 MB/s eta 0:00:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=44f1f05e437771cde2c71494c2e79cbb4049036460810bc09763fdd9bc985cb6
  Stored in directory: /Users/nadyashechtman/Library/Caches/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
  Created wheel for imageio-ffmpeg: filename=imageio_ffmpeg-0.5.1-py3-none-any.whl size=167

In [31]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 250 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 4.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [34]:
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [69]:
##start OCR(Get Frames)

import cv2

video_path = 'VideoEZ.MP4'

vidcap = cv2.VideoCapture(video_path)

frame_count = 0
saved_count = 0
frame_interval = 10  # Save every 5th frame (adjust as needed)

success, image = vidcap.read()

while success:
    if frame_count % frame_interval == 0:
        cv2.imwrite(f"frame{saved_count}.jpg", image)
        saved_count += 1  # Increment the count of saved frames
    success, image = vidcap.read()
    frame_count += 1  # Increment the count of total frames


In [70]:
print(frame_count)

4782


In [71]:
##READ OCR

import pytesseract
from PIL import Image


final_set = set()
final_dict = {}
ID = 0
actual_range = frame_count//frame_interval


for i in range(actual_range):
    frame_path = f'frame{i}.jpg'
    img = Image.open(frame_path)
    text = pytesseract.image_to_string(img)
    final_set.add(text)
    final_dict[text] = ID
    ID+=1
    print(f"Text in frame {i}: {text}")


Text in frame 0: 1900 @CGCEC8e0

Attendees Q @

Eytan Gipi, West Coast
San Diego State University (SDSU)

Ezra Galperin, Tri-State
Cornell University

Ezra Rubin, West Coast
University of California, Davis (UCD)

Fluer Hassan-Nahoum
Deputy Mayor of Jerusalem

Freddi Rappoport, Southeast
University of Florida (UF)

Gabriel Benghiat
Director of Student and Alumni Mark...
Hillel International

Gabriel Gaysinsky
University of California, Davis (UCD)

Gabriel Silverstein, Midwest
University of Michigan

Text in frame 1: Attendees One

Eytan Gipi, West Coast
San Diego State University (SDSU)

Ezra Galperin, Tri-State
Cornell University

Ezra Rubin, West Coast
University of California, Davis (UCD)

Fluer Hassan-Nahoum
Deputy Mayor of Jerusalem

Freddi Rappoport, Southeast
University of Florida (UF)

Gabriel Benghiat
Director of Student and Alumni Mark...
Hillel International

Gabriel Gaysinsky
University of California, Davis (UCD)

Gabriel Silverstein, Midwest
University of Michigan


Text in

In [8]:
names = """1. Bar Kador
2. Barry Klein - Co-Founder, Kinetic Campaigns
3. Beaue Bernstein - University of Pennsylvania
4. Becca Erikson - University of Cincinnati
5. Becca Carp - Vanderbilt University
6. Ben Lahyani - University of Minnesota
7. Ben Messafi - University of Pennsylvania
8. Ben Stern - Rutgers University New Brunswick
9. Eliana Jaffa - University of Florida (UF)
10. Elie Gaffen - University of Illinois at Urbana-Champaign
11. Elijah Feldman - University of California, Berkeley
12. Elisabeth Rosenfeld - Chief Field Officer, Israel on Campus Coalition
13. Elisha Baker - Columbia University
14. Eliza Krantz - Leadership Development Program, AIPAC
15. Elizabeth Lunzer
16. Dena Shink - Indiana University
17. Deondray Radford - The Ohio State University
18. Devin Kornblum - University of Tennessee, Knoxville
19. Dillon Meyer - Photographer
20. Dion J. Pierre - Campus Correspondent, Algemeiner
21. Christian Scandrett - Point Loma Nazarene University
22. Claire Conner - Northwestern University
23. Clara Calavia Sarnago - University of Florida (UF)
24. Cole Bennett - University of Florida (UF)
25. Corey Silberglied - University of Miami
26. Cyrus Gomes - University of Alabama
27. Cyrus Sabet - Florida State University (FSU)
28. Dorothy Tananbaum - Board Chair, Israel on Campus Coalition
29. Einat Wilf - Former Member of Knesset
30. Elan Shoichet - Brooklyn College
31. Elena Yacov - Adam and Gila Milstein Family Foundation
32. Eli Fisher - Fordham Law School
33. Elia Groode - University of Southern California (USC)
34. Elyse Martin - Director of Alumni Engagement, Passages Israel
35. Emily Austin - Journalist and Podcast Host
36. Emily Bell - University of Florida (UF)
37. Emily Kahan - Columbia University
38. Emma Beigelman - University of South Florida (USF)
39. Emma Burke - Syracuse University
40. Emma Platt - Washington University in St. Louis
41. Eva Czegledi - Syracuse University
42. Evan Cohen - University of Michigan
43. Eve Kirshner - Emory University
44. Evelyn Eggers - Midwest Regional Director, Israel on Campus Coalition
45. Eyal Shechtman - University of Washington
46. Eytan Gipi - San Diego State University (SDSU)
47. Avi Gordon - Alums for Campus Fairness
48. Avi Mayer - Editor-in-Chief
49. Avi Posnick - StandWithUs
50. Aviel Seltzer - Queens College
51. Avital Strauss - Brown University
52. Aviva Jacobs - Schusterman Family Philanthropies
53. Aviva Klompas - Co-Founder, Boundless Israel
54. Andrew Leubitz
55. Andrew Sabadash - Southeast, University of Florida (UF)
56. Andrew Tananbaum - Israel on Campus Coalition
57. Andrew Turquie - West Coast, University of Southern California
58. Alyssa Konesky - Michigan State University
59. Alyza Lewin - President, The Brandeis Center, Inc.
60. Ambra Caserta - New York, Pace University
61. Amina Abdukarimova - Baruch College
62. Amjad Taha - Journalist and Political Advisor
63. Ana Laura Leyser - Midwest, University of Iowa
64. Andrei Mandelshtam - Stanford University
65. Andrew Fox - Research Fellow, Henry Jackson Society
66. Andrey Pikovskiy - Midwest, Loyola University Chicago
67. Angelina Palumbo - Tri-State, SUNY-New Paltz
68. Anna Langer - Schusterman Family Philanthropies
69. Ari Feinsmith - West Coast, University of California, Davis (UCD)
70. Ari Willis - Northeast, Emerson College
71. Ari Wisch - Senior Counselor to the Chairwoman, Committee on Education & the Work
72. Ariane de Gennaro - Tri-State, Yale University
73. Arianna Brown - Northeast, Wellesley College
74. Dafna Slepkov - The Ohio State University
75. Dakota Hutchison - Southeast, University of Oklahoma
76. Eyal Yakoby - University of Pennsylvania
77. Ezra Galperin - Tri-State, Cornell University
78. Ezra Rubin - West Coast, University of California, Davis (UCD)
79. Fluer Hassan-Nahoum - Deputy Mayor of Jerusalem
80. Freddi Rappoport - Southeast, University of Florida (UF)
81. Aidan Bloomstine - West Coast, University of Southern California
82. Aidan Herzlinger - New York, Baruch College
83. Ajay Kori - Co-Founder, Novilla Pharmaceuticals and UrbanSci
84. Alan Meyerson - Counsel, Israel on Campus Coalition
85. Alec Bachman - Williams College, The London School
86. Alex Belanger - Midwest, University of Illinois at Urbana-Champaign
87. Alex Nelson - Southeast, University of Florida (UF)
88. Alexa Greenstein - Southeast, University of South Florida (USF)
89. Alexa Morchelies - Southeast, University of Central Florida (UCF)
90. Alexa Schwartz - Midwest, University of Kansas
91. Alexander Greene - Tri-State, Yale University
92. Alexander Walter - West Coast, Arizona State University (ASU)
93. Alexandra Ahdoot - Duke University
94. Alicia Fusco - Tristate Deputy Regional Director, Israel on Campus Coalition
95. Alisa Rosenblat - Midwest, Washington University in St. Louis
96. Allie Shisgal
97. Allison Frank - Midwest, Purdue University
98. Allon Sepiashvili - Midwest, Indiana University
99. Adrian Balbus - Baruch College
100. Adriana Garcia Cruz - Southeast, Florida International University (FIU)
101. Adina Fink - Digital Producer, Israel on Campus Coalition
102. Adina Navon - Charles and Lynn Schusterman Family Philanthropies
103. Adina Thompson - University of Kansas
104. Adira Greenwald - New York, Binghamton University
105. Ariella Ross - New York, Queens College
106. Arron Critchfield
107. Artie Mackey - Towson University
108. Arvind Asokan - West Coast, Stanford University
109. Aryeh Kalb - CUNY Graduate Center
110. Asaf Friedler - West Coast, University of San Francisco (USF)
111. Audrey Zeff - Southeast, Georgia Institute of Technology
112. Ben Shalom Elhadad - Johns Hopkins University (JHU)
113. Benjamin Alkon - Staff Attorney, The Louis D. Brandeis Center for Human Rights Under Law
114. Benjamin Comire - CUFI on Campus
115. Benjamin Levy - Northeast, Brown University
116. Benjamin Marcus - Northeast, Brown University
117. Benjamin Meppen - New York University
118. Benjamin Reznikov - West Coast, University of California, San Diego (UCSD)
119. Bethany Mandel - Co-Founder, Shield Communications PR
120. Bridget Lewis - Israel Advocacy Group
121. Bridgette Meshkani - University of California, Berkeley (UC)
122. Brooklynn Romine - Midwest, Indiana State University
123. Bryan Alpert
124. Carlos Vazquez - Oral Roberts University
125. Carly Cooperman - CEO, Schoen Cooperman Research
126. Carol Black - Feature Group
127. Carol Kornworcel - Geller International Fellow, Brandeis University
128. Carrie Talansky - Kirsh Philanthropies
129. Cassidy Mcgoldrick
130. Chance Dutton - THE Ohio State University
131. Charles Covit
132. Chris Townsend - Photographer
133. Daniel White - Breakout Tech, 46 Entertainment
134. Danielle Nagad - Texas State University - San Marcos
135. Danielle Sobkin - University of California, Berkeley (UC)
136. Danielle Weitzel - Indiana State University
137. Danielle Yablonka - Florida Atlantic University (FAU)
138. David Caplan - Temple University
139. David Donin - New York, Baruch College
140. David Gregory - Rutgers University New Brunswick
141. David Lederer - Tri-State, Columbia College
142. David Lender - Northeast, University of Delaware
143. David Marias - National Political Director, Israel on Campus Coalition
144. David Mayofis - Digital Influencer, Israel on Campus Coalition
145. David McCrudden - Southeast, The University of North Carolina at Chapel Hill
146. David Ndayishimiye - Midwest, University of Missouri, Columbia
147. David Rittberg - Senior Director for U.S. Jewish Grantmaking, Charles and Lynn Schusterman Family Philanthropies
148. David Rodriguez - Oral Roberts University
149. David Roumani
150. David Shapiro
151. David Swindle - Jewish News Syndicate
152. Debbie Engelberg - Brandeis University
153. Debbie Fuehrer - Clinical Psychologist and Assistant Professor, Mayo Clinic
154. Debi Ghate - Partner, Steadfast
155. Lucy Silverman - Assistant Director of Heritage Education, Zeta Beta Tau
156. Luda Isakharov - University of Oregon
157. Luke Prest - Southeast, Louisiana State University
158. Lydia Buckius - Southeast, University of Miami
159. Lynn Schusterman - Founder and Chair Emerita, Schusterman Family Philanthropies
160. Mackenzie Cooper - Tri-State, Binghamton University
161. Macklin Lapp - Midwest, Indiana University
162. Madeleine Stern - Northwestern University
163. Jonus Goldstein - Southeast, Florida State University (FSU)
164. Jordan Eskew - University of San Diego
165. Jose Suriel
166. Josef Kay - Northeast, Brandeis University
167. Joseph Zebede - Southeast, Tulane University
168. Josh Harrison - Los Angeles Valley College
169. Leo Kalb Bourke - West Coast, Pomona College
170. Leora Feinsmith - West Coast, California Polytechnic State University
171. Rabbi Levi Shemtov
172. Liel Shlafrok - Midwest, DePaul University
173. Lielle Lasko - New York, Hunter College
174. Lillian (Lilli) Russ - Tri-State, Rutgers University New Brunswick
175. Sadie Saxe - New York, Stony Brook University
176. Sam Ben-Ur
177. Sam Fingerhut - The Ohio State University
178. Sam Mitrani
179. Samantha Berlan - Duke University
180. Samantha Cavaliere - New York, Baruch College
181. Samantha Gaines - Southeast, The University of North Carolina at Chapel Hill
182. Sami Berkoff - National, National Initiatives
183. Miguel Burger-Calderon
184. Mikayla Feldman - Northeast, Virginia Tech
185. Minaleah Koffron - Midwest, Michigan State University (MSU)
186. Miriam Cherson - New York, The City College of New York
187. Miriam Elman - Executive Director, AEN
188. Miryam Hersh - Midwest, University of Michigan
189. Mitch Eisen
190. Romie Avivi Stuhl - West Coast, University of Oregon
191. Rose Ritch - Northeast Regional Director, Israel on Campus Coalition
192. Rowan Ostrowicki - Southeast, University of Texas at Dallas
193. Russell Robinson - CEO, Jewish National Fund
194. Rusty Rosenthal - Jewish Federation of Greater Washington
195. Ryan Alberts - University of Minnesota
196. Ryan Kassanoff - ICC Senior Geller International Fellow
197. Rick Abrams - Israel Programs Admissions Director, Jewish National Fund-USA
198. Rick Fromberg - BerlinRosen
199. Riley Mehall - Butler University
200. Rina Cohen - UJA-Federation New York
201. Rishon Dev Netala - Arizona State University (ASU)
202. Riva Rubin - New York, Columbia University
203. RJ Richman - Chief Digital Officer, Israel on Campus Coalition
204. Jake Rosen - Brandeis University
205. Jake Stone - West Coast, University of Colorado, Boulder
206. James Scher - West Coast, University of California, Santa Cruz
207. Jared Berelowitz - Northeast, Northeastern University
208. Jason Fink - Southeast, Elon University
209. Jason Goldschmidt - University of Chicago
210. Mateusz Karpow - Midwest, Ball State University (BSU)
211. Matisyahu - Musician
212. Matthew Cohen - West Coast, University of California, Irvine (UCI)
213. Matthew Gaynor - Southeast, Florida International University (FIU)
214. Matthew Kotcher - Northeast, Brown University
215. Matthew Lodge - Miami University, Ohio
216. Matthew Zivian - Midwest, Michigan State University (MSU)
217. Max Fletcher - West Coast, Sonoma State University
218. Marc Rowan - CEO and Co-Founder, Apollo Global Management
219. Maria Viramontes - University of Texas at San Antonio
220. Marian Stern - Principal, Projects in Philanthropy
221. Marilyn Rosenthal - Director, AIPAC PAC
222. Marjan Greenblatt - Founder and Director, Stop Femicide Iran
223. Molly Seghi - Southeast, University of Florida (UF)
224. Morgan Ames - Emory University
225. Morgan Pogoda - Southeast, University of South Florida (USF)
226. Moshe Janani - New York, Brooklyn College
227. Noah Rubin - University of Pennsylvania
228. Noah Slepkov - NZS
229. Noga Gottlieb - West Coast, University of San Francisco (USF)
230. Ofer Shalom - Director of Training, Community Security Service
231. Opal Cohen - Northeast, Virginia Tech
232. Oren Abutbul - West Coast, Santa Clara University
233. Steve O'Dell - Israel on Campus Coalition
234. Suzanne Gershowitz - TPESF
235. Sydney Grossman - Southeast, Vanderbilt University
236. Tal Appel - Tri-State, School of General Studies - Columbia University
237. Tali Dardashtian - New York, New York University (NYU)
238. Talia Nazarian - Northeast, George Washington University (GWU)
239. Tamir Hanono-Michli - University of Wisconsin-Madison
240. Yocheved Rothschild - Northeast, University of Maryland, College Park
241. Yoel Mayerfeld - Chase Properties
242. Yoella Peterman Rosenzweig - Israel on Campus Coalition
243. Yoni Lalouz - Deputy West Coast Regional Director, Israel on Campus Coalition
244. Yosef Wolchansky - West Coast, Arizona State University (ASU)
245. Yotam Polizer - CEO, IsraAID
246. Zach Astrof - Northeast, University of Maryland, College Park
247. Zach Leslie - New York University
248. Gabrielle Guigui - Northeast, George Washington University (GWU)
249. Gabrielle Ostad - Emory University
250. Gage Bernstein - Other, Temple University
251. Garrett Sabol - Northeast, Temple University
252. Gidon Halbfinger - The Paul E. Singer Foundation
253. Grant Baxter - Southeast, The University of Tennessee
254. Gregory Gotlieb - Northeast, University of Massachusetts Amherst
255. Greta Loewenthal - University of Michigan
256. Gustavo Trope Mayer - Northeast, Brown University
257. Jeff Miller - National Operations Director, Israel on Campus Coalition
258. Jeff Sklar - Co-Chairman, Sklar Kirsh
259. Jeremy Kazzaz - Beacon Coalition
260. Jill Myers - Integrative Nutrition Health Coach, Yodaya Wellness
261. Jillian Arnold - Midwest, University of Iowa
262. Jillian Kessel - Midwest, Miami University, Ohio
263. Joah Levine - Southeast, University of Florida (UF)
264. Jon Sutton - Partner, Sutton and Smart Campaigns
265. Josh Mellody-Pizzato - Western States Director, Israel on Campus Coalition
266. Joy Amidror - Northeast, American University (AU)
267. Judah Guggenheim - Tri-State, Princeton University
268. Jules Ahdoot - Southeast, Duke University
269. Julia Abbott - Dartmouth College
270. Julia Schulman - Paul E. Singer Foundation
271. Kaitlyn Rodriguez - Southeast, University of South Florida (USF)
272. Keren Azaria
273. Lauren Samidi - West Coast, San Diego State University (SDSU)
274. Lauren Schor Geller - The Geller Family Foundation
275. Lauren Suriel - National Field Director, Israel on Campus Coalition
276. Lauren Williamson - Fulbright Scholar, Geller International Intern, Connecticut College
277. Layla Modlin - New York, Binghamton University
278. Lea Speyer - Program Officer, Maimonides Fund
279. Lea Visher - Southeast, University of Florida (UF)
280. Logan Fraigun - West Coast, University of California, Riverside (UCR)
281. Logan Schoen - American University (AU)
282. Lorraine Sanchez - Northeast, Salem State University
283. Michael Oren - Former Israeli Ambassador to the U.S.
284. Mitch Wolf - Tri-State, Rutgers University New Brunswick
285. Mitchell Eisen
286. Moshe Lencer - Director of Campus Affairs, American Jewish Committee
287. Naftali Bennett - Former Prime Minister of the State of Israel
288. Nathan Rosenberg - Midwest, University of Michigan
289. Nathaniel Moldoff - The Snider Foundation
290. Nathaniel Stein - Tri-State, SUNY - University of Albany
291. Nechama Kittner - Midwest, Marquette University
292. Netanel Crispe - Tri-State, Yale University
293. Nicolas Hernandez - West Coast, California State University, Fullerton
294. Nicole Hirschkorn - Southeast, Emory University
295. Nikolay Remizov - Midwest, Saint Louis University
296. Peri Karpishpan - Northeast, Tufts University
297. Pnina Sasson - Southeast, Tulane University
298. Rabbi Akiva Berkowtz - Mashgiach
299. Rachel Applefield - Indiana University
300. Rachel Bohnik - New York, Queens College
301. Rachel Cusnir - University of Michigan
302. Rachel Shapiro - Salem State University
303. Raeefa Shams - Director of Communications and Programming, Academic Engagement Network
304. Raphael Englander - Northeast, University of Pennsylvania
305. Raphael Escoboza - Social Media Influencer
306. Raquel (Kelly) Shapiro - Rutgers University
307. Rayna Handis - Midwest, United Democracy Project Intern, The Ohio State University
308. Rebecca Brumer - Midwest, Marquette University
309. Rebecca Izkiyayeva - New York, Brooklyn College
310. Rebecca Michaeli - Northeast, Wellesley College
311. Rebecca Shafron - Schusterman Family Philanthropies
312. Rebekah Mann - Director of Campus Programming and Initiatives, Students Supporting Israel
313. Remi Harmolin - Tri-State, Rutgers University New Brunswick
314. Rennie Cohen - Southeast, University of Oklahoma
315. Rich Goldberg - Senior Advisor, Foundation for Defense of Democracies
316. Rich Rayant
317. Richard Priem - CEO, Community Security Service
318. Robert Augustin - Zeta Beta Tau
319. Rochel Leah Bernstein - Co-Founder and CEO, Spark Family Offices
320. Shabbos Kestenbaum - Shield Communications Fellow, Shield Communications PR
321. Shahar Cohen - West Coast, Arizona State University (ASU)
322. Shana Adelpour - University of California, Los Angeles (UCLA)
323. Shane Silverman - Midwest, Marquette University
324. Shani Menna - West Coast, University of California, San Diego (UCSD)
325. Shani Mizrahi - New York, Yeshiva University
326. Shannon Saed - New York, The City College of New York
327. Shari Merrill - Hillel International
328. Sharon Knafelman - University of California, Berkeley (UCB)
329. Shira Rosenberg - Midwest, Northwestern University
330. Shira Wasser - Chief Leadership Officer, ICC
331. Shoham Sadovsky
332. Sholom Miller - Musician
333. Shuli Frenkel - Northeast, University of Maryland, College Park
334. Shye Klein - Photographer and Survivor
335. Sophia Pisarevskiy - New York, Hunter College
336. Sophia Solomon - West Coast, University of San Francisco (USF)
337. Sophia Witt - Executive Vice President, Students Supporting Israel
338. Sophie Kalmin - Emory University
339. Stella Vayner - New York, Columbia University
340. Theo Morgan - Southeast, Emory University
341. Thom Waye - Chief Strategy Officer, Chabad on Campus International
342. Thomas Ashley - Southeast, Georgia State University
343. Tia Geri - West Coast, Stanford University
344. Tim Cohen - Hillel International
345. Tim Mellman - Midwest, Washington University in St. Louis
346. Tim Wilbanks - LED Tech, 46 Entertainment
347. Toby Volcovics - Northeast, Boston University
348. Todd Silberglied - Director of Research and Analysis, Israel on Campus Coalition
349. Tomer Britva - UTSA - The University of Texas at San Antonio
350. Tomer Elias - CFO, Israel on Campus Coalition
351. Tyler Porter - West Coast, University of California, San Diego (UCSD)
352. Uriel Appel - Northeast, University of Maryland, College Park
353. Victor Chudowsky - TrueScore Consulting
354. Victoya Smith - Alabama A&M University
355. Vivian Hamui - Northeast, Brandeis University
356. Warren Weissbluth - Southeast, Rice University
357. William Eisen - Northeast, George Mason University (GMU)
358. William Wallier - West Coast, Occidental College
359. Willie Williams - Office Manager, Israel On Campus Coalition
360. Xander Pinto - New York, Binghamton University
361. Yael Danon - Northeast, Harvard University
362. Yael Fine - Midwest, Indiana University - Bloomington
363. Yael Friedkin - Senior Director of External Relations, Community Security Initiative
364. Zachary Ainbinder-Barkley - American University (AU)
365. Zachary Kogen - Midwest, Indiana University
366. Zachary Patterson - Duke University
367. Zachary Sadoff - Midwest, University of Minnesota
368. Zack Thompson - Midwest, University of Illinois at Chicago
369. Zoe Mail - Deputy Southeast Regional Director, Israel on Campus Coalition
370. Zoe Siegel - Midwest, The Ohio State University
"""

In [9]:
import re 

result = re.split(r'\d+.', names)

# Remove newline characters from each element in the list and exclude empty strings
result = [res.replace("\n", "").strip() for res in result if res.strip()]

# Split each element by the first hyphen
result = [res.split('-', 1) for res in result]



print(result)
len(result)


[['Bar Kador'], ['Barry Klein ', ' Co-Founder, Kinetic Campaigns'], ['Beaue Bernstein ', ' University of Pennsylvania'], ['Becca Erikson ', ' University of Cincinnati'], ['Becca Carp ', ' Vanderbilt University'], ['Ben Lahyani ', ' University of Minnesota'], ['Ben Messafi ', ' University of Pennsylvania'], ['Ben Stern ', ' Rutgers University New Brunswick'], ['Eliana Jaffa ', ' University of Florida (UF)'], ['Elie Gaffen ', ' University of Illinois at Urbana-Champaign'], ['Elijah Feldman ', ' University of California, Berkeley'], ['Elisabeth Rosenfeld ', ' Chief Field Officer, Israel on Campus Coalition'], ['Elisha Baker ', ' Columbia University'], ['Eliza Krantz ', ' Leadership Development Program, AIPAC'], ['Elizabeth Lunzer'], ['Dena Shink ', ' Indiana University'], ['Deondray Radford ', ' The Ohio State University'], ['Devin Kornblum ', ' University of Tennessee, Knoxville'], ['Dillon Meyer ', ' Photographer'], ['Dion J. Pierre ', ' Campus Correspondent, Algemeiner'], ['Christian S

372

In [38]:
import pandas as pd

df = pd.DataFrame(result, columns=['Names', 'Affiliation'])

df['Affiliation'] = df['Affiliation'].fillna("")

df['linkedin'] = ""
print(df)

names_list = df["Names"].tolist()

             Names Affiliation linkedin
0  Slava Shechtman   Company A         
1   Eyal Shechtman   Company B         
2        Ron Hoory   Company C         
3       Moshe Levi                     


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

final_name_dict = {}
driver = webdriver.Chrome()

driver.get("https://www.linkedin.com/login")

time.sleep(2)

email_field = driver.find_element(By.ID, "username")
email_field.send_keys("eyal.shechtman@gmail.com") 

password_field = driver.find_element(By.ID, "password")
password_field.send_keys("Toftof1209!") 

login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
login_button.click()

time.sleep(5)
for name in names_list:
    search_bar = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
    search_bar.clear()
    search_bar.send_keys(name)  
    search_bar.send_keys(Keys.RETURN)  

    time.sleep(5)

    first_profile = driver.find_element(By.XPATH, "(//a[contains(@href, '/in/')])[1]")
    first_profile.click() 

    profile_url = driver.current_url
    final_name_dict[name] = profile_url
    
    print("Profile URL:", profile_url)

# Close the browser
driver.quit()

Profile URL: https://www.linkedin.com/search/results/all/?keywords=slava%20shechtman&origin=GLOBAL_SEARCH_HEADER&sid=lNs
Profile URL: https://www.linkedin.com/in/eyal-shechtman/
Profile URL: https://www.linkedin.com/search/results/all/?keywords=Ron%20Hoory&origin=GLOBAL_SEARCH_HEADER&sid=G%2C_
Profile URL: https://www.linkedin.com/search/results/all/?keywords=Moshe%20Levi&origin=GLOBAL_SEARCH_HEADER&sid=T%3AG


In [33]:
import pandas as pd

# List of names
fakeData = ["Slava Shechtman", "Eyal Shechtman", "Ron Hoory", "Moshe Levi"]

# Corresponding affiliations
Affi = ["Company A", "Company B", "Company C", "Company D"]

# Create the DataFrame
df = pd.DataFrame({
    'Name': fakeData,
    'Affiliation': Affi
})

# Initialize the LinkedIn column
df["LinkedIn"] = ""

# Add LinkedIn URLs using a loop
id = 1
for name in fakeData:
    df.loc[df['Name'] == name, "LinkedIn"] = "linkedin.com/" + str(id)
    id += 1

# Display the DataFrame
print(df)

              Name Affiliation        LinkedIn
0  Slava Shechtman   Company A  linkedin.com/1
1   Eyal Shechtman   Company B  linkedin.com/2
2        Ron Hoory   Company C  linkedin.com/3
3       Moshe Levi   Company D  linkedin.com/4
